# QKD BB84
## Summary

```
for i = 1..n {
  // Alice
  with a new random bit b {
    with a random polarisation p selected from R or D {

             0°                     R = + (rectilinear)
             |                      D = X (diagonal)
             |
      --------------- 90°
             |
             |

      if (p == R)
        if (b == 0) Alice -----Photon at 0° -----> Bob
        if (b == 1) Alice -----Photon in 90° -----> Bob
      if (p == D)
        if (b == 0) Alice -----Photon at 45° -----> Bob
        if (b == 1) Alice -----Photon at 135° -----> Bob
    }
    Record b,p

    // Eve
    Possibly try to read the photon

    // Bob
    with a random polarisation p selected from R or D {
      measure photon p as r
      record p,r
    }
  }
}

Bob -----polarisation used for each bit (public)-----> Alice
Alice -----Which polarisations were correct-----> Bob

// Alice and Bob
Remove the bits that were read with the wrong polarisation

Alice -----Selected keystring bits (public)-----> Bob
Bob -----match/mismatch (public)-----> Alice

if match, both Alice and Bob remove the test key bits from their keystring, resulting in a shared key.

```

In [1]:
import math, random

In [77]:
NumberOfPhotons = 1000
Peavesdrop = 0.01

AliceBase = []
AliceKey = []
BobBase = []
BobKey = []
SharedKey = []

for i in range(NumberOfPhotons):
  # Alice
  AliceKey.append(random.randint(0,1))
  AliceBase.append(random.randint(0,1))
  if   AliceBase[-1] == 0 and AliceKey[-1] == 0: p = 0
  elif AliceBase[-1] == 0 and AliceKey[-1] == 1: p = 90
  elif AliceBase[-1] == 1 and AliceKey[-1] == 0: p = 45
  elif AliceBase[-1] == 1 and AliceKey[-1] == 1: p = 135

  #print(f'Alice sends a {AliceKey[-1]} in basis {AliceBase[-1]} with {p} polarisation')

  # Eve
  if random.random() < Peavesdrop: # Chance Eve tries to read this photon
    p = random.randint(0,1)
    if p == 1: p = 45

  # Bob
  BobBase.append(random.randint(0,1)) # 0 is 0° (rectilinear), 1 is 45° (diagonal)

  if   BobBase[-1] == 0 and p ==   0: BobKey.append(0)
  elif BobBase[-1] == 0 and p ==  45: BobKey.append(random.randint(0,1))
  elif BobBase[-1] == 0 and p ==  90: BobKey.append(1)
  elif BobBase[-1] == 0 and p == 135: BobKey.append(random.randint(0,1))
  elif BobBase[-1] == 1 and p ==   0: BobKey.append(random.randint(0,1))
  elif BobBase[-1] == 1 and p ==  45: BobKey.append(0)
  elif BobBase[-1] == 1 and p ==  90: BobKey.append(random.randint(0,1))
  elif BobBase[-1] == 1 and p == 135: BobKey.append(1)

  #print(f'Bob reads a   {BobKey[-1]} in basis {BobBase[-1]}')

# Bob sends Alice his BobBase array
# Alice responds with a list of entries where the wrong basis was used

# Alice and Bob
AliceKey2 = []
BobKey2 = []
for i in range(NumberOfPhotons):
  if AliceBase[i] == BobBase[i]:
    AliceKey2.append(AliceKey[i])
    BobKey2.append(BobKey[i])

print(f'Key length with wrong basis errors removed: {len(AliceKey2)}')

Key length with wrong basis errors removed: 494


Intrusion detection

In [78]:
FractionOfKeyToVerify = 0.1
# Alice
KeyCheckIndex = [random.randint(0,len(AliceKey2)-1) for x in range(math.floor(len(AliceKey2)*FractionOfKeyToVerify))]
KeyCheck = []
for i in range(len(KeyCheckIndex)): KeyCheck.append(AliceKey2[KeyCheckIndex[i]])

# Alice transmits KeyCheckIndex and KeyCheck to Bob over public channel

# Bob
KeyIsValid = True
for i in range(len(KeyCheckIndex)):
  if KeyCheck[i] != BobKey2[KeyCheckIndex[i]]:
    KeyIsValid = False
    break

# Bob tells Alice if everything matched

# Alice and Bob remove the check bits from their shared key
if KeyIsValid:
  SharedKey = []
  for i in range(len(AliceKey2)):
    if i not in KeyCheckIndex:
      SharedKey.append(AliceKey[i])

  print(f'Shared key[{len(SharedKey)}]: {SharedKey}')

else:
  print('Intruder detected, no key was generated')

Shared key[447]: [1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0